In [1]:
import numpy as np

In [37]:
def get_firms_ids(cursor):
  query = "select id from firms offset 3000 limit 10"
  cursor.execute(query)
  return cursor.fetchall()

def compute_geo_firms(id):
  query = """
    with
    envelope as (
      select ST_MakeEnvelope(
        ST_X(ST_Project(firms.point, 1000, radians(90))::geometry), 
        ST_Y(ST_Project(firms.point, 1000, radians(180))::geometry), 
        ST_X(ST_Project(firms.point, 1000, radians(-90))::geometry),
        ST_Y(ST_Project(firms.point, 1000, radians(0))::geometry),
        4326
      ) as geom, firms.point as point, date, brightness, brightness_t31, radiative_power
      from firms where id = %(id)s
    ),
    roads as (
      select ST_Union(ST_Intersection(envelope.geom, ukraine_roads.geom)) as geom
      from ukraine_roads, envelope
      where ST_Intersects(envelope.geom, ukraine_roads.geom)
    ),
    rails as (
      select ST_Union(ST_Intersection(envelope.geom, ukraine_railways.geom)) as geom
      from ukraine_railways, envelope
      where ST_Intersects(envelope.geom, ukraine_railways.geom)
    ),
    settlements as (
      select population, ukraine_settlements.geom as geom, name,
      ST_Distance(ukraine_settlements.geom::geography, envelope.point::geography) as distance
      from ukraine_settlements, envelope
      where population > 0
      order by distance asc
      limit 10
    ),
    center as (
      select ARRAY_AGG(population) as population, ARRAY_AGG(distance) as distance from settlements
    )
    select 
    	ST_X(envelope.point) as lon, ST_Y(envelope.point) as lat,
      date, brightness, brightness_t31, radiative_power,
      ST_Distance(roads.geom::geography, envelope.point::geography) as dist_road,
      ST_Distance(rails.geom::geography, envelope.point::geography) as dist_rail,
      population, distance
    from roads, rails, envelope, center
  """
  def execute(cursor):
    cursor.execute(query, { "id": id })
    return cursor.fetchall()
    
  return execute

In [48]:
from DBEngine import db_engine

firms = db_engine.execute(get_firms_ids)
for entry in firms:
  geo = db_engine.execute(compute_geo_firms(entry[0]))[0]
  geo = geo[:8] + (np.mean(np.array(geo[8]) / ((np.array(geo[9]) ** (1/2)) + 1)),)
  print(f"id {str(entry[0])}: {str(geo)}")

id 3001: (36.7716, 47.909, datetime.date(2021, 7, 16), 302.0, 291.5, 6.3, 319.85526072, None, 4.7463761704619545)
id 3002: (33.4385, 47.8731, datetime.date(2021, 7, 16), 304.6, 293.3, 4.1, 160.79158514, 44.13766776, 887.4355903293175)
id 3003: (30.6275, 48.164, datetime.date(2021, 7, 16), 302.7, 284.0, 5.4, 560.61788907, 618.88624689, 18.345407964324487)
id 3004: (30.6141, 48.1619, datetime.date(2021, 7, 16), 320.1, 286.8, 18.7, 267.66441539, 1.40085317, 21.609056434977298)
id 3005: (37.5975, 47.0999, datetime.date(2021, 7, 16), 307.5, 296.4, 9.6, 50.25190954, 80.71025044, 762.188420588241)
id 3006: (34.2716, 50.0018, datetime.date(2021, 7, 16), 303.2, 290.3, 5.7, 948.87788891, None, 3.3303360799413717)
id 3007: (34.2854, 49.9999, datetime.date(2021, 7, 16), 301.2, 290.3, 4.5, 492.07627511, None, 3.7314483878568923)
id 3008: (37.7058, 48.1671, datetime.date(2021, 7, 16), 302.4, 292.2, 6.3, 42.67795509, 66.18015291, 55.14150945439602)
id 3009: (34.9793, 48.4892, datetime.date(2021, 7, 1